In [1]:

####################################################################################################
####################################################################################################

import sys
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras.layers import Dense
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

sys.path.append(os.path.abspath("../"))
from helpers import *

sys.path.append(os.path.abspath("../../"))
from pax_utils import s1s2_utils

from get_dnn_data import *


Using TensorFlow backend.


In [9]:

####################################################################################################
####################################################################################################
    
nEventsFile     = 25000
nEventsTrain    = 20000
iEventStart     = 0
iEventEnd       = iEventStart + nEventsFile

s2_window_max   = 2300
resample_factor = 46
resample_factor = 230

n_timesteps     = int(s2_window_max / resample_factor)

assert(s2_window_max % resample_factor == 0)

file_out_input = 'train/array_train_input_events%06d-%06d_timesteps%04d' % (iEventStart, iEventEnd - 1, n_timesteps) + '.npy'
file_out_truth = 'train/array_train_truth_events%06d-%06d_timesteps%04d' % (iEventStart, iEventEnd - 1, n_timesteps) + '.npy'

file_out_input = 'data_all.npy'
file_out_truth = 'truth_all.npy'

train_data  = np.load(file_out_input)
train_truth = np.load(file_out_truth)

#train_data  = train_data [nEventsTrain:, :]
#train_truth = train_truth[nEventsTrain:, :]


nEventsTrain = 100000

train_data  = train_data [nEventsTrain:, :]
train_truth = train_truth[nEventsTrain:, :]

print()
print(train_data.shape)
print(train_truth.shape)
print()


####################################################################################################
####################################################################################################

model_name_h5 = 'models/'

#model_name_h5 += 'model_model_xy_s2waveforms_dnn_elu_acc9902_epochs10_layers-1270-1270-127-2'
model_name_h5 += 'dnn_xy_s2waveforms_elu_acc9915_epochs10_layers-1270-1270-127-2'

model_name_h5 += '.h5'

file_out = './predictions/' + os.path.basename(model_name_h5).replace('.h5', '.pkl')

print(file_out)



(20967, 1270)
(20967, 2)

./predictions/dnn_xy_s2waveforms_elu_acc9915_epochs10_layers-1270-1270-127-2.pkl


In [10]:

####################################################################################################
# Predict
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(train_data)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")



Loaded Model: models/dnn_xy_s2waveforms_elu_acc9915_epochs10_layers-1270-1270-127-2.h5
Predicted



In [11]:

####################################################################################################
####################################################################################################

arr_x_true = train_truth[:, 0]
arr_y_true = train_truth[:, 1]

arr_x_pred = arr_xy_pred[:, 0]
arr_y_pred = arr_xy_pred[:, 1]

arr_dx     = arr_x_true - arr_x_pred
arr_dy     = arr_y_true - arr_y_pred


print(arr_x_pred.shape)
print(arr_y_pred.shape)
print(arr_x_true.shape)
print(arr_y_true.shape)



(20967,)
(20967,)
(20967,)
(20967,)


In [12]:

####################################################################################################
####################################################################################################

df_out = pd.DataFrame()

df_out['x_pred'] = arr_x_pred
df_out['y_pred'] = arr_y_pred

df_out['x_true'] = arr_x_true
df_out['y_true'] = arr_y_true


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[0:5][:])
print()


,x_pred,y_pred,x_true,y_true
0,10.528956,-5.394522,11.452923,-4.799238
1,17.457815,-14.849562,19.004622,-14.180182
2,-0.398164,-30.419701,-0.713852,-30.766731
3,-30.271709,-17.710238,-31.193504,-19.178736
4,14.097367,-12.268551,15.701762,-12.451200
